Trabalho realizado por: Bárbara Freixo, PG49169


Esta implementação implementa uma classe que define a lógica para o algoritmo de classificação Prism. Ela possui métodos para inicializar os atributos, encontrar as regras de classificação, avaliar uma regra, verificar se uma regra cobre um dado, retornar a classe majoritária num conjunto de dados, realizar a classificação de novos dados e representar a classe como string. O método "fit" é o responsável por encontrar as regras de classificação, ele funciona enquanto houver dados para serem classificados, procurando a melhor regra em cada iteração e adicionando-a à lista de regras. O método "predict" é o responsável por realizar a classificação de novos dados, ele percorre cada dado e aplica as regras encontradas na iteração do método "fit" até encontrar uma regra que cobre o dado, caso nenhuma regra cobra o dado, a classe majoritária dos dados iniciais é retornada.
        

In [2]:
class Prism:
    # Método construtor que inicializa os atributos da classe Prism
    def __init__(self, data, target, attributes):
        # Conjunto de dados
        self.data = data
        # Atributo alvo
        self.target = target
        # Lista de atributos
        self.attributes = attributes
        # Lista de regras
        self.rules = []
        # Classe majoritária nos dados
        self.default_class = self.majority_class(data)

    # Método fit que encontra as regras de classificação
    def fit(self):
        # Enquanto houver dados para serem classificados
        while len(self.data) > 0:
            # Inicializa as variáveis para armazenar a melhor regra
            best_rule = None
            best_coverage = 0
            best_accuracy = 0

            # Percorre todos os atributos
            for attribute in self.attributes:
                # Percorre todos os valores possíveis para o atributo
                for value in set([row[attribute] for row in self.data]):
                    # Cria a regra
                    rule = (attribute, value)
                    # Avalia a regra
                    coverage, accuracy = self.evaluate_rule(rule)

                    # Verifica se a regra é a melhor até o momento
                    if (coverage, accuracy) > (best_coverage, best_accuracy):
                        best_rule = rule
                        best_coverage = coverage
                        best_accuracy = accuracy

            # Se não foi encontrada nenhuma regra, encerra o loop
            if best_rule is None:
                break

            # Adiciona a regra à lista de regras
            self.rules.append((best_rule, self.majority_class(self.data)))

            # Remove dos dados aqueles que foram cobertos pela regra
            self.data = [row for row in self.data if not self.rule_covers(row, best_rule)]

    # Método que avalia uma regra
    def evaluate_rule(self, rule):
        # Dados cobertos pela regra
        covered = [row for row in self.data if self.rule_covers(row, rule)]
        # Acurácia da regra
        accuracy = sum(row[self.target] == self.majority_class(covered) for row in covered) / len(covered) if len(covered) > 0 else 0
        # Retorna o número de dados cobertos e a acurácia da regra
        return len(covered), accuracy

    # Método que verifica se uma regra cobre um dado
    def rule_covers(self, row, rule):
        attribute, value = rule
        return row[attribute] == value

    # Método que retorna a classe majoritária num conjunto de dados
    def majority_class(self, data):
        # Se não houver dados, retorna a classe majoritária dos dados iniciais
        if len(data) == 0:
            return self.default_class
        # Obtém a lista de classes para cada dado
        classes = [row[self.target] for row in data]
        # Retorna a classe que aparece com mais frequência
        return max(set(classes), key=classes.count)
    
    # Método que realiza a classificação de novos dados
    def predict(self, data):
        # Lista para armazenar as previsões
        predictions = []
        # Percorre cada dado
        for row in data:
            # Percorre cada regra
            for rule, target_class in self.rules:
                # Verifica se a regra cobre o dado
                if self.rule_covers(row, rule):
                    # Adiciona a classe prevista à lista de previsões
                    predictions.append(target_class)
                    break
            else:
                # Se nenhuma regra cobre o dado, adiciona a classe majoritária dos dados iniciais
                predictions.append(self.majority_class(self.data))
        # Retorna a lista de previsões
        return predictions

    # Método que representa a classe como string
    def __repr__(self):
        # Lista para armazenar as strings das regras
        rule_strings = [f"IF {attribute} == {value} THEN class = {target_class}" for (attribute, value), target_class in self.rules]
        # Junta as strings das regras com quebra de linha
        return "\n".join(rule_strings)

# Testes e exemplos para o algoritmo implementado

## Exemplo de uso do código

Esta implementação é um exemplo de como usar o classificador Prism num dataset. Ele carrega o dataset Iris, divide os dados em conjuntos de treino e teste, treina o classificador Prism com os dados de treino e preve as classes nos dados de teste. Finalmente, ele calcula a precisão do classificador usando a função accuracy_score da biblioteca sklearn. O resultado é exibido na tela com as regras do classificador e a precisão.

In [4]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Carrega o dataset Iris
iris = datasets.load_iris()

# Converte os dados numa lista de dicionários
data = [
    {attribute: value for attribute, value in zip(iris.feature_names, row)}
    for row in iris.data
]
target = 'class'

# Adiciona a classe alvo a cada ponto de dados
for row, target_value in zip(data, iris.target):
    row[target] = target_value

# Divide os dados em conjuntos de treino e teste
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Separa as características da classe alvo nos dados de teste
test_data_no_target = [{k: v for k, v in row.items() if k != target} for row in test_data]

# Instancia e treina o classificador Prism
attributes = iris.feature_names
prism = Prism(train_data, target, attributes)
prism.fit()

# Preve as classes nos dados de teste
predictions = prism.predict(test_data_no_target)

# Calcula a precisão do classificador
y_true = [row[target] for row in test_data]
accuracy = accuracy_score(y_true, predictions)

print("Prism classifier rules:")
print(prism)
print(f"\nAccuracy: {accuracy:.2f}")

Prism classifier rules:
IF petal width (cm) == 0.2 THEN class = 1
IF sepal width (cm) == 3.0 THEN class = 1
IF petal width (cm) == 1.3 THEN class = 1
IF petal width (cm) == 1.0 THEN class = 2
IF sepal length (cm) == 6.3 THEN class = 2
IF sepal width (cm) == 2.8 THEN class = 2
IF sepal width (cm) == 3.2 THEN class = 2
IF sepal width (cm) == 3.1 THEN class = 2
IF petal width (cm) == 0.4 THEN class = 0
IF sepal width (cm) == 2.7 THEN class = 2
IF petal width (cm) == 0.3 THEN class = 2
IF sepal width (cm) == 2.5 THEN class = 2
IF sepal width (cm) == 3.3 THEN class = 2
IF sepal width (cm) == 2.6 THEN class = 2
IF petal width (cm) == 0.1 THEN class = 2
IF sepal length (cm) == 6.0 THEN class = 2
IF sepal width (cm) == 2.9 THEN class = 2
IF sepal length (cm) == 5.0 THEN class = 2
IF sepal length (cm) == 7.7 THEN class = 2
IF sepal length (cm) == 5.5 THEN class = 2
IF sepal length (cm) == 6.2 THEN class = 2
IF sepal length (cm) == 7.2 THEN class = 2

Accuracy: 0.37


## Testes "Unittest"

Foi realizado um teste utilizando o unittest para testar a implementação do PRISM. Temos então o seguinte teste:

In [5]:
import unittest

class TestPrism(unittest.TestCase):
    #Test the functionality of the Prism classifier
    def test_prism_classifier(self):
        data = [
            {'outlook': 'sunny', 'temperature': 'hot', 'humidity': 'high', 'wind': 'weak', 'class': 'no'},
            {'outlook': 'sunny', 'temperature': 'hot', 'humidity': 'high', 'wind': 'strong', 'class': 'no'},
            {'outlook': 'overcast', 'temperature': 'hot', 'humidity': 'high', 'wind': 'weak', 'class': 'yes'},
            {'outlook': 'rain', 'temperature': 'mild', 'humidity': 'high', 'wind': 'weak', 'class': 'yes'},
            {'outlook': 'rain', 'temperature': 'cool', 'humidity': 'normal', 'wind': 'weak', 'class': 'yes'},
            {'outlook': 'rain', 'temperature': 'cool', 'humidity': 'normal', 'wind': 'strong', 'class': 'no'},
            {'outlook': 'overcast', 'temperature': 'cool', 'humidity': 'normal', 'wind': 'strong', 'class': 'yes'},
            {'outlook': 'sunny', 'temperature': 'mild', 'humidity': 'high', 'wind': 'weak', 'class': 'no'},
            {'outlook': 'sunny', 'temperature': 'cool', 'humidity': 'normal', 'wind': 'weak', 'class': 'yes'},
            {'outlook': 'rain', 'temperature': 'mild', 'humidity': 'normal', 'wind': 'weak', 'class': 'yes'},
            {'outlook': 'sunny', 'temperature': 'mild', 'humidity': 'normal', 'wind': 'strong', 'class': 'yes'},
            {'outlook': 'overcast', 'temperature': 'mild', 'humidity': 'high', 'wind': 'strong', 'class': 'yes'},
            {'outlook': 'overcast', 'temperature': 'hot', 'humidity': 'normal', 'wind': 'weak', 'class': 'yes'},
            {'outlook': 'rain', 'temperature': 'mild', 'humidity': 'high', 'wind': 'strong', 'class': 'no'},
        ]
        attributes = ['outlook', 'temperature', 'humidity', 'wind']
        target = 'class'
        prism = Prism(data, target, attributes)

        # Test rule creation
        prism.fit()
        self.assertGreater(len(prism.rules), 0, "No rules were created.")

        # Test prediction
        test_data = [{'outlook': 'sunny', 'temperature': 'cool', 'humidity': 'normal', 'wind': 'weak'}]
        predictions = prism.predict(test_data)
        expected_predictions = ['yes']
        self.assertEqual(predictions, expected_predictions, "Incorrect predictions.")

        # Test accuracy
        data_no_target = [{k: v for k, v in row.items() if k != target} for row in data]
        predictions = prism.predict(data_no_target)
        y_true = [row[target] for row in data]
        accuracy = accuracy_score(y_true, predictions)
        self.assertGreater(accuracy, 0, "Incorrect accuracy calculation.")

        # Test __repr__ method
        repr_result = repr(prism)
        self.assertIsInstance(repr_result, str, "Incorrect __repr__ result.")

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


Esta implementação testa o funcionamento do classificador Prism. Ela cria uma instância do classificador Prism usando um conjunto de dados de exemplo e conjuntos de atributos e variáveis alvo. A implementação então testa a criação de regras, previsões, precisão e a representação em string do classificador.